In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None


In [2]:
from galaxy_datasets import gz2,gz_decals_5  # or gz_hubble, gz_candels, ...

gz2_catalog_train, _ = gz2(
    root='data/gz2',
    train=True,
    download=False
)

gz2_catalog_test, _ = gz2(
    root='data/gz2',
    train=False,
    download=False
)

gzd5_catalog_train, _ = gz_decals_5(
    root='data/gzd5',
    train=True,
    download=False
)

gzd5_catalog_test, _ = gz_decals_5(
    root='data/gzd5',
    train=False,
    download=False
)

In [209]:
# buta parsing https://oup.silverchair-cdn.com/oup/backfile/Content_public/Journal/mnras/471/4/10.1093_mnras_stx1829/1/stx1829_Supp.zip?Expires=1714517667&Signature=Rxbs5olh38b8ZQuHpIO5wzvbC0pYPAgTV6cjs094XQ46xNQYrwEBeFRnM2j4075C-8fb2QzNZ2DxVvzVzpqwSip~qThZNQ7McML2XBAEEhPSKqjYEBq6reaT6fVoibyA3y2PA2~-kg3~Gd5fuTIh~sCgRKA~adjGDdlgzrwhsGsRgIpGVU5zTFIW9y~Fp2NAqkvAngvLZmcOrLiGZ-m21HtgEMTaFQf0tXaNRxz9hK08hcU7hm6YVJwG8Bj7kjgWlZLqEeWm6t6FjjRRBrbpE3827VmO2JQcoavHODbBTPz33OVORetuDIIFy53IsIPCYtkzl9NYZg921XdNteiyMQ__&Key-Pair-Id=APKAIE5G5CRDK6RD3PGA
galaxy_list = []

f = open("buta_list.txt", "r")
i = 1
for line in f:
    line = line.strip()
    if(len(line)>0):
        if (line[0].isupper()):
            splitted = [s.strip() for s in line.split('&')] 
            if(len(splitted)>1):
                if(splitted[0] not in ('Other name','Other')):
                    g_name = splitted[0]
                    if('.' not in g_name and g_name[0]=='J'):
                        parts = g_name.split('+')
                        if len(parts) == 2:
                            g_name = parts[0][:7] + '.' + parts[0][7:] + '+' + parts[1][:6] + '.' + parts[1][6:]
                        
                    galaxy_list.append([g_name,splitted[1],splitted[2],i])

    i+=1

galaxy_df = pd.DataFrame(galaxy_list, columns = ['name','pgc' ,'img_name','row_txt']) 
galaxy_df.img_name = galaxy_df.img_name.astype(int)
hyperleda = pd.read_csv('hyperleda.txt')
print('source: ', 3962)
print('parsed: ',len(galaxy_df))
print('found in hyperleda: ',len(hyperleda)) #found in hyperleda base



source:  3962
parsed:  3960
found in hyperleda:  3727


In [223]:
gz2_zenodo = pd.read_csv('data/gz2-zenodo/gz2_filename_mapping.csv') #https://zenodo.org/records/3565489#.Y3vFKS-l0eY
gz2_zenodo_rings = gz2_zenodo.merge(galaxy_df, left_on ='asset_id', right_on = 'img_name')
gz2_zenodo_rings.to_csv('data/target-mining/gz2_zenodo_rings.csv',index = False)

In [237]:
#https://github.com/mwalmsley/galaxy-datasets/tree/main
gz2_train_iauname = gz2_catalog_train.merge(gz2_zenodo_rings, left_on = 'id_str', right_on = 'objid' )
gz2_test_iauname = gz2_catalog_test.merge(gz2_zenodo_rings, left_on = 'id_str', right_on = 'objid' )
gz2_rings = pd.concat([gz2_train_iauname,gz2_test_iauname])
gz2_rings.to_csv('data/target-mining/gz2_rings.csv',index = False)

In [241]:
gzd5_train_iauname = gzd5_catalog_train.merge(gz2_rings, on = 'iauname' )
gzd5_test_iauname = gzd5_catalog_test.merge(gz2_rings, on = 'iauname' )
gzd5_rings = pd.concat([gzd5_train_iauname,gzd5_test_iauname])
gzd5_rings.to_csv('data/target-mining/gzd5_rings.csv',index = False)

In [3]:
import platform

print(platform.processor())

i386


In [4]:
gzd5_rings.shape

NameError: name 'gzd5_rings' is not defined